# Peak selection

From chromatin accessibility anndata (peaks called), select peaks for downstream QTL testing and other tasks.

Inputs:
- Select ca-peaks

Outputs:

Notes:
- Source of peak selection should change to here.

## Setup

In [ ]:
import os
import sys
# import json
# import gc
# from datetime import date
# import numpy as np
# import pandas as pd
# import anndata as ad
# import pyBigWig

In [2]:
# Ensure cwd is project root and that /code is in path

import os
import sys

cwd = os.path.basename(os.getcwd())

if cwd == 'footprintQTL':
    
    PROJECT_DIR = '.'


elif cwd == 'code':
    
    PROJECT_DIR = '..'


elif cwd == 'fichtner':

    PROJECT_DIR = 'projects/footprintQTL'

else:

    PROJECT_DIR = 'manual'


os.chdir(PROJECT_DIR)
sys.path.append(os.getcwd() + '/code')

In [3]:
from helpers.python.utils import create_dir, list_files_and_links, list_dirs, read_rownames

## Variables

In [13]:
# User variables

from glob_vars import \
                    MAIN_ENV, \
                    DATASET, CT_MAP_ID, \
                    SNP_LOCS_BED, SELECT_PEAKS_TSV_DIR, SELECT_TEST_REGIONS_DIR, CHROM_SIZES \

scales = (15, 25, 51, 101)
radii = tuple(scale // 2 for scale in scales)


In [6]:
os.environ['PATH'] = f'/omics/groups/OE0540/internal_temp/users/fichtner/micromamba/envs/{MAIN_ENV}/bin:' + os.environ['PATH']
from pybedtools import BedTool

# Load data

In [9]:
snps_bed = BedTool(SNP_LOCS_BED)

## Variant-centred peaks

In [ ]:
IN_DIR = SELECT_PEAKS_TSV_DIR
in_dirs = list_dirs(IN_DIR)

for cell_type in in_dirs:

    print(cell_type)
    q_dir = f'{IN_DIR}/{cell_type}'


    caqtl_bed = BedTool(f'{q_dir}/peaks_ca-qtls.bed')


    ## Modalities

    ## ca-qtl

    # Save: test regions
    tregion_caqtl_bed_out = os.path.join(SELECT_TEST_REGIONS_DIR, CT_MAP_ID, cell_type, 'test-regions_ca-qtls.bed')
    create_dir(tregion_caqtl_bed_out)
    caqtl_bed.saveas(tregion_caqtl_bed_out)

    # Save: test regions pm1k
    caqtl_pm1k_bed = caqtl_bed.slop(b=1000, g=CHROM_SIZES)

    tregion_caqtl_p1mk_bed_out = os.path.join(SELECT_TEST_REGIONS_DIR, CT_MAP_ID, cell_type, 'test-regions_ca-qtls_pm1k.bed')
    create_dir(tregion_caqtl_p1mk_bed_out)
    caqtl_pm1k_bed.saveas(tregion_caqtl_p1mk_bed_out)


    for radius, scale in zip(radii, scales):

        ## vc-Peaks

        # Save: test regions
        tregions_vc_bed_out = os.path.join(SELECT_TEST_REGIONS_DIR, CT_MAP_ID, cell_type, f'test-regions_ca-qtls_variant-centred_{scale}bp.bed')
        create_dir(tregions_vc_bed_out)
        caqtl_bed.saveas(tregions_vc_bed_out)

        # vc-Peaks
        snps_overlap = snps_bed.intersect(caqtl_bed, wa=True)
        peaks_vc_bed = snps_overlap.slop(b=radius, g=CHROM_SIZES)

        # Save: vc-Peaks
        peaks_vc_bed_out = os.path.join(SELECT_PEAKS_TSV_DIR, cell_type, f'peaks_ca-qtls_variant-centred_{scale}bp.bed')
        create_dir(peaks_vc_bed_out)
        peaks_vc_bed.saveas(peaks_vc_bed_out)


        ## vc-Peaks +-1kbp

        # Save: test regions
        tregions_pm1kb_vc_bed_out = os.path.join(SELECT_TEST_REGIONS_DIR, CT_MAP_ID, cell_type, f'test-regions_ca-qtls_pm1k_variant-centred_{scale}bp.bed')
        create_dir(tregions_pm1kb_vc_bed_out)
        caqtl_pm1k_bed.saveas(tregions_pm1kb_vc_bed_out)

        # vc-Peaks
        snps_overlap = snps_bed.intersect(caqtl_pm1k_bed, wa=True)
        peaks_vc_pm1kb_bed = snps_overlap.slop(b=radius, g=CHROM_SIZES)

        # Save: vc-Peaks
        peaks_pm1kb_vc_bed_out = os.path.join(SELECT_PEAKS_TSV_DIR, cell_type, f'peaks_ca-qtls_pm1k_variant-centred_{scale}bp.bed')
        create_dir(peaks_pm1kb_vc_bed_out)
        peaks_vc_pm1kb_bed.saveas(peaks_pm1kb_vc_bed_out)

DL-EN
Glia
UL-EN
Neural-progenitors
Midbrain-EN
